# Convolutions Aren't Convoluted!

Convolution is a basic mathematical operation -- in a very real way, it's only slightly less fundamental than addition and multiplication\*! Because of its fundamental nature, convolution arises repeatedly in both theoretical and applied contexts.

This notebook focuses on the mathematical foundations of convolution. If you're interested in the applications of convolution, check out the [second notebook in this tutorial](02%20-%20Why%20Are%20Convolutions%20Useful%3F.ipynb).

Below, I assume pretty minimal math background. There is a very elegant view of convolutions that comes from the theory of Fourier transforms and linear algebra. If you're interested, check out [this nice exposition of that point of view](http://www.neurotheory.columbia.edu/Ken/math-notes/math-notes-4.pdf), from Kenneth Miller of Columbia, which does a good job developing the approach gently. I'm a neuroscientist, so I like that there are lots of neuroscience examples!

\* For the mathematically inclined: convolutions are defined whenever you have a set with a binary operation -- that means groups, monoids, and even categories! Check out [this blog post by Chris Olah](http://colah.github.io/posts/2014-12-Groups-Convolution/) for more on that front, or for a very nice introduction to groups.

## Preliminaries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from matplotlib import rcParams

%matplotlib inline
plt.xkcd();
rcParams['font.sans-serif'] = ['Comic Sans','TSCu_Comic']

from scipy.stats import multivariate_normal

from scipy.io import wavfile
from IPython.display import Audio

import scipy.signal

import utils.util as util

# Understanding Signals

In the traditional view, convolutions arise when we want to understand how very simple systems respond when signals pass through them. In order to make this idea concrete, we need to define "simple systems" and "signals".

Discrete signals are, fundamentally, just indexed collections of numbers, also known as arrays. When we have a signal in time, a signal is an array of numbers. When that signal is distributed in space, like an image, it is a 2-D array of numbers.

Below, we generate a signal using a [random walk](https://en.wikipedia.org/wiki/Random_walk). Random walks are used to model everything from the stock market to the motion of atoms. They represent the simplest form of an [auto-correlated](https://en.wikipedia.org/wiki/Autocorrelation) signal, so they're a nice modest step up from [white noise](https://en.wikipedia.org/wiki/White_noise). We generate a random walk by adding together samples from a Gaussian distribution -- the value at time `t` is just the running total of the sum of `t` samples.

Because this signal is random, it can be helpful to come back and re-run the cell below in order to get more examples later -- some examples will be better than others for illustrating points about convolution below.

In [ ]:
signals = [util.randomWalk(tMax=0.1,eps=0.01)]
signal = signals[0]
signalName = 's'

util.plotSignal(signal,signalName)

Since our signal is just an array, we can think of it as a collection of points. The zeroth element of the array is a point at $t = 0$, the next is a point at $t = 1$, etc. The height of the point is determined by the number in the array.

Put another way, we can break a signal of length $N$ down into $N$ components. If we were to represent the component at the timepoint $i$ as a function $e_i$, it would look like:

$$
e_i(t) = 1 \text{ if } t = i \\
e_i(t) = 0 \text{ otherwise }
$$

This function is also called a "Kronecker's Delta", "the delta function", or the "unit impulse". If we were to draw it, it might look like:

In [ ]:
util.plotKronecker()

We construct our signal from these components as follows: we multiply each $e_i$ by the signal $s$. The elements that are $0$ stay zero, while the element that is $1$ now has the value $s(i)$. We then add all $N$ of the $e_i$ together, and the result is our original signal.

If you've taken linear algebra, this process may sound familiar to you -- it's exactly the way that we construct a vector out of a set of basis vectors. This is a deep connection. Just as the canonical, or usual, basis set for a two-dimensional vector space is the set $\left\{[1, \ 0], [0,\ 1]\right\}$, one common basis set for signals is the set of all Kronecker delta functions, each at a different point in time or space. 

In [ ]:
util.plotSignalAsDelta(signals[0],'s',color='b')

# Understanding Simple Transformations

For us, a simple transformation is one whose response to a signal is just the sum of its responses to all of the components in the signal and which doesn't depend on time. In mathematical terms, it is *linear* and *translation-invariant*. Since the components of our signals are just single points at different times, this means we can learn everything we want to learn about a simple transformation by seeing what it does to a single point at a single time. 

This response is called a *kernel* or an *impulse response*. If we put in two points at once, the output is just two kernels stacked on top of one another -- note that this is the same thing as multiplying by 2. If we put in one point, then wait one time step and put in a second point, we get two copies of the kernel, separated by one time step.

Let's take a look at some simple transformations. We start by defining and plotting their kernels.

In [ ]:
# Define Our Kernels
pad = 15; padding = [0]*pad
delta = [1]+padding
delay = padding+delta
echo = [1]+padding+[1]+padding

kernels = [delta,delay,echo]
kernelNames = ['f',"f'","(f+f')"]
numKernels = len(kernels)

In [ ]:
# Plot Our Kernels  
util.kernelsPlot(kernels,kernelNames)
plt.suptitle('Kernels!',fontsize=20,weight='bold',y=1.1);

Now, lets apply these kernels to our "unit impulse" -- one point. Notice what comes out: it's just the kernel!

In [ ]:
simpleSignal = [1]
simpleSignals = [simpleSignal]

for kernel in kernels:
    simpleSignals.append(np.convolve(simpleSignal,kernel))

util.convolutionPlot(simpleSignals,'just one point',kernels,kernelNames)

The "unit" in "unit impulse" means "having height 1". What happens if we scale our unit impulse so that it has a different height?

The response is just a scaled version of the kernel, just as when we put in two points at the same time.

In [ ]:
simpleSignal = [1/2]
simpleSignals = [simpleSignal]

for kernel in kernels:
    simpleSignals.append(np.convolve(simpleSignal,kernel))

util.convolutionPlot(simpleSignals,'just one point',kernels,kernelNames)

Notice that the third filter's kernel is just the sum of the first two kernels. The response of the third filter is also a sum: it's the sum of the responses to the first two kernels.

This is also true if we put two signals in: the response to a sum of inputs is just the sum of the responses to the individual inputs.

So if we have a more complicated signal, what we need to do to get the response to that signal is combine scaled, time-shifted copies of the kernel.

Put more formally: any signal can be broken up into scaled delta functions at different times, and the response of our simple transformation to a scaled delta function at some time $t$ is just a scaled copy of the kernel starting at that time $t$.

Let's put that into mathematical terms. 

$$
\begin{align}
\text{response}_\text{filter}(\text{signal}) =& \sum_t \text{response}_\text{filter}(\text{signal}(t)) \\
    =& \sum_t \text{response}_\text{filter}(\text{impulse at time $t$}) \cdot \text{signal}(t)
\end{align}
$$

We can read this from left to right and recover our original description in English: the response of a filter to a signal is given by the sum of responses to impulses, each at a different time $t$, scaled by the value of the signal at the time $t$.

In mathematics, we try to avoid writing out long expressions like the above, since they're a pain to manipulate. So let's "simplify" our notation, even though it makes it harder to remember what's going on. We'll call the signal $s$ and the response of the filter $r_f$. We'll call the response to an impulse at time $t$ $k_t$. That makes our statement above:

$$
r_{f}(s) = \sum_t k_t \cdot s(t)
$$

This gives us the overall response -- the new signal that comes out of our filter -- but what if we wanted to know the response at a particular time $t$? In order to do that, we have to keep track of all the possible contributions to the response at that time. That is, the response at $t$ will include the beginning of the response to the point at time $t$ plus the ongoing responses to the points at time $t-1$, at time $t-2$, and so on. We'll call that time difference $\Delta$. 

And what is the value of the response at $t$ to the input at $t-\Delta$? If $\Delta$ is $0$, then it's the zeroth\* element of the kernel, scaled by $s(t)$. If $\Delta$ is $1$, then it's the first element of the kernel, scaled by $s(t-1)$. If $\Delta$ is $2$, then it's the second element of the kernel, scaled by $s(t-2)$.

There's an obvious pattern here: the response at $t$ to the input at $t-\Delta$ is just the $\Delta$th element of the kernel times the value of the signal at $t-\Delta$.

We should give a name to $t-\Delta$. The traditional one is $\tau$.

To reiterate: the response to a point at time $\tau$ is going to be a $s(\tau)$-scaled copy of the kernel response. At any given time $t$ after $\tau$, there will usually be several copies stacked on top of each other, each of them at a different point in their response, depending on how long after $\tau$ the time $t$ is. This time difference is $\Delta$.

Let's write that out mathematically:

$$
r_{f}(s)(t) = \sum_{\tau+\Delta = t} s(\tau) \cdot r(\Delta)
$$

where summing over $\tau+\Delta=t$ means summing over all pairs of $\tau$ and $\Delta$ that sum to $t$. This mathematical expression just says what we said above: the response of a filter to a signal is a group of time-shifted kernels scaled by the past values of the signal. This expression just narrows that down to a single point.

The mathematical expression above is exactly a convolution! It's not usually written that way, so for the benefit of anyone who has seen convolutions before, let's make some notation changes. First, we'll use the normal symbols: $f$ and $g$ instead of $r_f$ and $s$, and put a $*$ between them instead of putting parentheses. The result looks like

$$
g*f(t) = \sum_{\tau+\Delta = t} g(\tau) \cdot f(\Delta)
$$

It's also standard to use $\tau$ by itself, making use of the fact that $\Delta=t-\tau$:

$$
g*f(t) = \sum_{\tau} g(\tau)\cdot f(t-\tau)
$$

This is the more familiar expression of convolution, but I think it hides more than it shows. It's meant to evoke a dot product between the signal and a "flipped around" version of the kernel, but that ends up deeply confusing people. There's nothing "flipped around" about the kernel at all -- it's just that the later elements of the signal are contributing the earliest part of their responses. That is, as we look backwards in the signal, we find components that have made it further through their repsonse.

\* Using Python notation

# Convolution for Filtering

The kernels we used above were somewhat contrived: they mostly just repeated the signal. What if we wanted to do something more useful?

Below, we define two more useful kernels: one that takes the difference between two points, and another that takes their average.

In [ ]:
# Define our filter kernels
pad = 1;  padding = [0]*pad
difference = padding+[1/2,0,-1/2]+padding;
average = padding+[1/2,1/2]+padding

kernels = [difference,average]
kernelNames = ['difference','average']
numKernels = len(kernels)

In [ ]:
# Plot Our Kernels
util.kernelsPlot(kernels,kernelNames)    
plt.suptitle('More Kernels!',fontsize=20,weight='bold',y=1.1);

The `average` kernel has two points, both at height $1/2$. Let's plug that definition into our convolution expression:

$$
\begin{align}
r_{f}(s)(t) &= \sum_{\tau+\Delta = t} s(\tau)\cdot r(\Delta) \\ \\
            &= s(t)\cdot r(0) + s(t-1)\cdot r(1) \\ \\
            &= s(t)\cdot 1/2 + s(t-1)\cdot 1/2 \\ \\
            &= \frac{s(t) + s(t-1)}{2}
\end{align}
$$

Notice that the final line is an expression for the average of two points. When we convolve this kernel with a signal, we get a *moving average*. How would we do a three-point moving average? What about an $N$-point moving average?

The `difference` kernel also has two points, but one is at height $1/2$ and the other is at height $-1/2$, and there's one at height $0$ in between. Let's plug that definition into our convolution expression:

$$
\begin{align}
r_{f}(s)(t) &= \sum_{\tau+\Delta = t} s(\tau)\cdot r(\Delta) \\ \\
            &= s(t)\cdot r(0) + s(t-1)*r(1) + s(t-2)\cdot r(2) \\ \\
            &= s(t)\cdot 1/2 + s(t-2)\cdot -1/2 \\ \\
            &= \frac{s(t) - s(t-2)}{2}
\end{align}
$$

Calculus aficionados will recognize that last line: it looks a lot like the definition of a derivative! It is the change in the signal divided by the change in time -- "the rise over the run". Convolving with this kernel gives us something like the first derivative of our signal -- though usually, we call it a *difference* to remind ourselves that we aren't dealing with infinite numbers of points. What do you think might happen if we applied this kernel repeatedly?

Below, we use our kernels on a signal like the one that we generated at the beginning of the section. You might be familiar with frequency-based analysis of signals. Can you see what our `difference` and `average` filters do to the signal in terms of frequency? Each time you run the code block below you'll get a new signal. Increasing the variable `tMax` will increase the length of the generated signal.

In [ ]:
signals = [util.randomWalk(tMax=0.5,eps=0.01)]

# Use Our Kernels
signals = [signals[0]]
for kernel in kernels:
    signals.append(np.convolve(signals[0],kernel))
    
util.convolutionPlot(signals,'signal',kernels,kernelNames)